In [2]:
# Convolutional neural network from scratch 
import cv2
import numpy as np 
import pandas as pd 
from matplotlib import pyplot as plt

In [3]:
data = pd.read_csv('mnist_train.csv')
data.head()

,label,1x1,1x2,1x3,1x4,1x5,1x6,1x7,1x8,1x9,...,28x19,28x20,28x21,28x22,28x23,28x24,28x25,28x26,28x27,28x28
0,5,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,4,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,9,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [22]:
data = np.array(data) # transform our csv into array 
lines, cols = data.shape # lines = 42000 / columns = 785 bcz the images are 28x28=784 pixels +1 with the beggining cols of label
np.random.shuffle(data) # because the train data are possibly sorted with a sequences of 0 then 1 then 2 and so on 
# we shuffe the dataset in order to avoid overfitting
data_validation = data[0:1000].T # Data transposition is performed in this particular context to facilitate 
# the manipulation of examples and features. Some machine learning algorithms and libraries prefer 
# this matrix representation with examples in columns rather than rows.
# data validation variable is used to evaluate and confirm the performance of our future model while training
label_val = data_validation[0]
result_val = data_validation[1:cols]

data_train = data[1000:lines].T
label_train = data_train[0] # we did the transpose so le first line is compose only by label
result_train = data_train[1:cols]

In [23]:
def init_params() :
    # W => weights / b => bias
    # 1st param : nb neurons on the actual layer / 2st param : nb neurons of the previous layer
    neuronsLayerOne = 10 
    neuronsLayerTwo = 10
    W1 = np.random.rand(neuronsLayerOne,cols-1) # -1 => to understand refer to what we said on the line of creation of cols
    b1 = np.random.rand(neuronsLayerOne,1) # which means for each neurons we generate a random number which will corresponding to the bias
    # bias doesn't have to be necessary between 0 and 1 ! 
    W2 = np.random.rand(neuronsLayerTwo,neuronsLayerOne)
    b2 = np.random.rand(neuronsLayerTwo,1)
    return W1, b1, W2, b2